In [ ]:
import numpy as np 
import pandas as pd
import warnings
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
warnings.simplefilter("ignore")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_validate, cross_val_predict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import seaborn as sns #for plotting
from sklearn.ensemble import RandomForestClassifier #for the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz #plot tree
from sklearn.metrics import roc_curve, auc #for model evaluation
from sklearn.metrics import classification_report #for model evaluation
from sklearn.metrics import confusion_matrix #for model evaluation
from sklearn.model_selection import train_test_split #for data splitting
import eli5 #for purmutation importance
from eli5.sklearn import PermutationImportance
import shap #for SHAP values
from pdpbox import pdp, info_plots #for partial plots
np.random.seed(123) #ensure reproducibility
pd.options.mode.chained_assignment = None  #hide any pandas warnings

In [ ]:
df = pd.read_csv("data33.csv")

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.corr()

In [ ]:
df['PATIENT_UPDATE'].value_counts() 

# The Explanation

In [ ]:
#Yellowbricks
from yellowbrick.features import Rank2D
import matplotlib.pyplot as plt

In [ ]:
visualizer = Rank2D(algorithm="pearson",  size=(1080, 720))
visualizer.fit_transform(X_train)
visualizer.poof()

In [ ]:
from yellowbrick.classifier import ClassificationReport
visualizer = ClassificationReport(model, size=(1080, 720))
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.poof()

In [ ]:
#Permutation Feature Importance：
perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
#LIME:LIME
#Lime
import lime.lime_tabular

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values,feature_names=X_train.columns.values.tolist(),class_names=y_train.unique())
predict_fn = lambda x: model.predict_proba(x).astype(float)
exp = explainer.explain_instance(X_test.values[0], predict_fn, num_features=35)
exp.show_in_notebook(show_all=False)

In [ ]:
#SHAP：
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values[1], X_test, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[1], X_test)

In [ ]:

def COVID19_risk_factors(model, patient):

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(patient)
    shap.initjs()
    return shap.force_plot(explainer.expected_value[1], shap_values[1], patient)

In [ ]:
data_for_prediction = X_test.iloc[1,:].astype(float)
COVID19_risk_factors(model, data_for_prediction)

In [ ]:
data_for_prediction = X_test.iloc[0,:].astype(float)
COVID19_risk_factors(model, data_for_prediction)

In [ ]:
shap_values = explainer.shap_values(X_train.iloc[:50])
shap.force_plot(explainer.expected_value[1], shap_values[1], X_test.iloc[:50])